In [1]:
import numpy as np

In [2]:
def fit(X_train, Y_train):
    result = {}
    class_values = set(Y_train) # keys

    for current_class in class_values:
        result[current_class]  = {}
        result["total_data"] = len(Y_train)
        
        current_class_rows = (Y_train == current_class)
        X_train_current = X_train[current_class_rows]
        Y_train_current = Y_train[current_class_rows]

        result[current_class]["total_count"] = len(Y_train_current) # total count of training data where current class is y = ai
        
        num_features = X_train.shape[1]
        for j in range(1, num_features+1):
            result[current_class][j]  = {}
            all_possible_values = set(X_train[:, j-1])# all possible values of j feature

            for current_value in all_possible_values:
                # gives count of training data where feature Xj = xj and class y = ai
                result[current_class][j][current_value] =( X_train_current[:, j-1]== current_value).sum() 

    return result

In [3]:
def predict(dictionary, X_test):

    y_pred = []
    for x in X_test:
        x_class = predictSinglePoint(dictionary,x)
        y_pred.append(x_class)
    return y_pred

In [4]:
def predictSinglePoint(dictionary,x):
    classes = dictionary.keys()

    best_p = -1000
    best_class = -1
    first_run = True
    
    for current_class in classes:
        if (current_class == "total_data"):
            continue
            
        p_current_class = probability(dictionary, x, current_class)
        if(first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False
    return best_class

In [5]:
def probability(dictionary, x, current_class):
    #output = dictionary[current_class]["total_count"] / dictionary["total_data"] # P(y = ai)
    output = np.log(dictionary[current_class]["total_count"]) - np.log(dictionary["total_data"]) # P(y = ai)

    num_features = len(dictionary[current_class].keys()) - 1 # subtract 1 due to one extra key of total features in that class

    for j in range(1,num_features+1):
        xj = x[j-1]
        count_current_class_with_value_xj = dictionary[current_class][j][xj] + 1 # plus 1 for laplace correction
        count_current_class = dictionary[current_class]["total_count"] + len(dictionary[current_class][j].keys()) # plus len for laplace correction

        #current_xj_probability = count_current_class_with_value_xj/count_current_class
        current_xj_probability = np.log(count_current_class_with_value_xj) - np.log(count_current_class)
        
        #output = output * current_xj_probability
        output = output + current_xj_probability

    return output

In [6]:
def makeLabelled(column):
    second_limit = column.mean()
    first_limit = 0.5 * second_limit
    third_limit = 1.5*second_limit
    for i in range (0,len(column)):
        if (column[i] < first_limit):
            column[i] = 0
        elif (column[i] < second_limit):
            column[i] = 1
        elif(column[i] < third_limit):
            column[i] = 2
        else:
            column[i] = 3
    return column

In [7]:
from sklearn import datasets
iris = datasets.load_iris()

In [8]:
X = iris.data
Y = iris.target

In [9]:
for i in range(0, X.shape[-1]):
    X[:, i] = makeLabelled(X[:,i])

In [10]:
from sklearn import model_selection
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X,Y, test_size = 0.25, random_state = 0)

In [11]:
dictionary = fit(X_train, Y_train)

In [12]:
Y_pred = predict(dictionary, X_test)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.94      1.00      0.97        16
           2       1.00      0.89      0.94         9

    accuracy                           0.97        38
   macro avg       0.98      0.96      0.97        38
weighted avg       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]


In [15]:
# GaussianNB is used with continuos data
iris = datasets.load_iris()
X = iris.data
Y = iris.target
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.25, random_state=0)

from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00         9

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38

[[13  0  0]
 [ 0 16  0]
 [ 0  0  9]]
